In [2]:
# 1️⃣ تثبيت المكتبات
!pip install -q torchaudio pydub rapidfuzz gTTS langdetect

# 2️⃣ استيراد المكتبات
import whisper
import os
from pydub import AudioSegment
from IPython.display import display, Javascript, Audio, HTML
from google.colab import output
import base64
import json
from rapidfuzz import fuzz
from gtts import gTTS
from langdetect import detect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.4 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [1]:
# 1️⃣ نتأكد إن النسخة القديمة اتشالت
!pip uninstall -y whisper
!pip uninstall -y openai-whisper

# 2️⃣ تثبيت النسخة الصح
!pip install -q openai-whisper

# 3️⃣ استيراد وتجربة
import whisper
model = whisper.load_model("small")
print("✅ Whisper شغال تمام")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 20.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


100%|███████████████████████████████████████| 461M/461M [00:17<00:00, 27.0MiB/s]


✅ Whisper شغال تمام


In [3]:
# 3️⃣ رفع ملفات القصص (عربي + إنجليزي)
from google.colab import files
print("✨ ارفع ملف stories_ar.json (قصص بالعربية)")
uploaded_ar = files.upload()
json_ar_path = list(uploaded_ar.keys())[0]

print("✨ ارفع ملف stories_en.json (قصص بالإنجليزية)")
uploaded_en = files.upload()
json_en_path = list(uploaded_en.keys())[0]


✨ ارفع ملف stories_ar.json (قصص بالعربية)


Saving stories_ar.json to stories_ar.json
✨ ارفع ملف stories_en.json (قصص بالإنجليزية)


Saving stories_en.json to stories_en.json


In [4]:
# 4️⃣ تحميل القصص
with open(json_ar_path, "r", encoding="utf-8") as f:
    stories_ar = json.load(f)

with open(json_en_path, "r", encoding="utf-8") as f:
    stories_en = json.load(f)

print(f"✅ تم تحميل {len(stories_ar)} قصة بالعربية و {len(stories_en)} قصة بالإنجليزية")


✅ تم تحميل 25 قصة بالعربية و 12 قصة بالإنجليزية


In [5]:
# 6️⃣ دالة لتحويل webm إلى wav
def convert_webm_to_wav(webm_file, wav_file):
    audio = AudioSegment.from_file(webm_file, format="webm")
    audio.export(wav_file, format="wav")

In [6]:
# 7️⃣ البحث في القصص
def search_stories_partial(query, lang="ar"):
    query = query.lower().strip()
    results = []

    # نحدد مصدر القصص
    data = stories_ar if lang == "ar" else stories_en

    for s in data:
        story_text = (s.get("content") or s.get("text") or "").lower()
        story_title = s.get("title", "").lower()

        # حساب التشابه
        score_title = fuzz.partial_ratio(query, story_title)
        score_text = fuzz.partial_ratio(query, story_text)

        # الأولوية للعنوان
        if score_title > 70:
            results.append((score_title + 20, s))
        elif score_text > 70:
            results.append((score_text, s))

    results.sort(reverse=True, key=lambda x: x[0])
    return [r[1] for r in results]


In [7]:

# 8️⃣ عرض القصة
def show_story(story, lang="ar"):
    title = story.get("title", "بدون عنوان")
    text = story.get("content") or story.get("text") or "لا يوجد نص"

    print(f"\n📖 {title}\n")
    print(text)

    # زر سماع القصة
    tts = gTTS(text, lang=lang)
    audio_file = "story_audio.mp3"
    tts.save(audio_file)
    display(Audio(audio_file, autoplay=False))

In [8]:
# 9️⃣ استقبال الصوت
def receive_audio(base64_string):
    audio_bytes = base64.b64decode(base64_string)
    webm_path = "recorded_audio.webm"
    with open(webm_path, "wb") as f:
        f.write(audio_bytes)
    wav_path = "recorded_audio.wav"
    convert_webm_to_wav(webm_path, wav_path)
    print(f"✅ تم حفظ الصوت كـ {wav_path}")

    # التعرف على النص واللغة
    result = model.transcribe(wav_path)  # auto-detect language
    text = result["text"].strip()
    lang = result["language"]  # "ar" أو "en"
    print(f"\n📢 نص التعرف الصوتي: {text}")
    print(f"🌍 تم التعرف على اللغة: {lang}\n")

    results = search_stories_partial(text, lang=lang)
    if results:
        print("✅ اختر من بين القصص القريبة:\n")
        for idx, r in enumerate(results, 1):
            title = r.get("title", "بدون عنوان")
            display(HTML(f"""
                <button onclick="google.colab.kernel.invokeFunction('notebook.show_story', [{idx-1}, '{lang}'], {{}})"
                        style="margin:5px;padding:10px 20px;font-size:18px;background:#2196F3;color:white;border:none;border-radius:8px;">
                    {title}
                </button>
            """))
        global last_results, last_lang
        last_results = results
        last_lang = lang
    else:
        print("❌ لم يتم العثور على قصة مطابقة.")


In [9]:

# 🔟 تسجيل الدوال
output.register_callback('notebook.receive_audio', receive_audio)
output.register_callback('notebook.show_story', lambda idx, lang: show_story(last_results[idx], lang))


In [11]:

# 1️⃣1️⃣ زر التسجيل
display(Javascript("""
function recordAudio() {
  navigator.mediaDevices.getUserMedia({audio:true}).then(stream => {
    const mediaRecorder = new MediaRecorder(stream);
    const audioChunks = [];
    mediaRecorder.ondataavailable = e => audioChunks.push(e.data);
    mediaRecorder.start();
    alert('🎤 سجل صوتك لمدة 5 ثوانٍ بعد الضغط على OK');
    setTimeout(() => { mediaRecorder.stop(); }, 5000);
    mediaRecorder.onstop = async () => {
      const audioBlob = new Blob(audioChunks);
      const arrayBuffer = await audioBlob.arrayBuffer();
      const base64String = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)));
      google.colab.kernel.invokeFunction('notebook.receive_audio', [base64String], {});
    };
  });
}

var button = document.createElement('button');
button.innerHTML = "🎙️ اضغط للتسجيل";
button.style.fontSize = "20px";
button.style.padding = "10px 20px";
button.style.background = "#4CAF50";
button.style.color = "white";
button.style.border = "none";
button.style.borderRadius = "10px";
button.onclick = recordAudio;
document.body.appendChild(button);
"""))

<IPython.core.display.Javascript object>

✅ تم حفظ الصوت كـ recorded_audio.wav

📢 نص التعرف الصوتي: محمد
🌍 تم التعرف على اللغة: ar

✅ اختر من بين القصص القريبة:




📖 قصة محمد صلى الله عليه وسلم

خاتم الأنبياء والمرسلين، وُلد في مكة يتيم الأب ثم الأم، ونشأ أمينًا صادقًا. بعثه الله في سن الأربعين برسالة الإسلام ليخرج الناس من عبادة الأصنام إلى عبادة الله وحده. لقي أذى كبيرًا من قريش، لكنه صبر وهاجر إلى المدينة حيث أسس دولة الإسلام. نشر العدل والرحمة، وجمع الناس على التوحيد. توفي في المدينة بعد أن أتم الله به الدين.


✅ تم حفظ الصوت كـ recorded_audio.wav

📢 نص التعرف الصوتي: Sekret Gordon.
🌍 تم التعرف على اللغة: pl

✅ اختر من بين القصص القريبة:




📖 Maya and the Secret Garden

Maya discovered a hidden garden behind an old wall. She watered the plants, and they grew into bright flowers. By caring for the garden, Maya found patience and joy in watching things grow.
